<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image_predictions.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image_predictions.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Prediction Import

* This notebook walks you through the process of uploading model predictions to a Model Run. This notebook provides an example for each supported prediction type for image assets. 

A Model Run is a container for the predictions, annotations and metrics of a specific experiment in your ML model development cycle.

**Supported annotations that can be uploaded through the SDK**

- Bounding box 
- Polygon
- Point
- Polyline 
- Classification free-text
- Classification - radio
- Classification - checklist




* Notes:
    * If you are importing more than 1,000 mask predictions at a time, consider submitting separate jobs, as they can take longer than other prediction types to import.
    * After the execution of this notebook a complete Model Run with predictions will be created in your organization. 

## Setup

In [ ]:
!pip install -q 'labelbox[data]'

In [ ]:
import labelbox as lb
import labelbox.data.annotation_types as lb_types
import labelbox.data.serialization as lb_serializers
import uuid
import numpy as np

## Replace with your API Key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY = None
client = lb.Client(API_KEY)

## Supported Predictions

In [ ]:
########### Radio Classification ###########

# Python annotation
radio_prediction = lb_types.ClassificationAnnotation(
    name="radio_question", 
    value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "second_radio_answer", confidence=0.5))
)

# NDJSON
radio_prediction_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer', 'confidence': 0.5}
} 

In [ ]:
########## Nested Classifications are only supported with NDJSON tools ##########

nested_radio_prediction_ndjson = {
  "name": "nested_radio_question",
  "confidence": 0.5 ,
  "answer": { "name": "first_radio_answer", "confidence": 0.5 },
      "classifications" : [
      {
        "name": "sub_radio_question", 
        "answer": {"name": "first_sub_radio_answer", "confidence": 0.5 }
      }
    ]
}


nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "confidence": 0.5 ,
  "answer": [{
      "name": "first_checklist_answer", 
      "confidence": 0.5,
      "classifications" : [
        {
          "name": "sub_checklist_question", 
          "answer": {"name": "first_sub_checklist_answer", "confidence": 0.5 }
        }          
      ]         
  }]
}



In [ ]:
############ Checklist ############

# Python Annotations
checklist_prediction = lb_types.ClassificationAnnotation(
  name="checklist_question", # must match your ontology feature's name
  value=lb_types.Checklist(
      answer = [
        lb_types.ClassificationAnswer(
            name = "first_checklist_answer", 
            confidence=0.5
        ), 
        lb_types.ClassificationAnswer(
            name = "second_checklist_answer", 
            confidence=0.5
        )
      ]
    )
 )

# NDJSON
checklist_prediction_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer' , 'confidence': 0.5},
    {'name': 'second_checklist_answer', 'confidence': 0.5}
  ]
}

In [ ]:
####### Bounding box #######


# Python Annotation 
bbox_prediction = lb_types.ObjectAnnotation(
  name = "bounding_box",  # must match your ontology feature's name
  confidence=0.5, 
  value=lb_types.Rectangle(
        start=lb_types.Point(x=977, y=1690), # Top left
        end=lb_types.Point(x=330, y=225), # Bottom right
    ),
  
)

#NDJSON 
bbox_prediction_ndjson = {
  'name': 'bounding_box', 
  'confidence': 0.5,
  'bbox': {
          "top": 977,
          "left": 1690,
          "height": 330,
          "width": 225
      }
}


In [ ]:
####### Bounding box with nested classification #######
bbox_with_radio_subclass_prediction = lb_types.ObjectAnnotation(
    name="bbox_with_radio_subclass",
    confidence=0.5, # must match your ontology feature's name
    value=lb_types.Rectangle(
        start=lb_types.Point(x=933, y=541), # Top left
        end=lb_types.Point(x=191, y=330), # Bottom right
    ),
    classifications=[
    	lb_types.ClassificationAnnotation(
        	name="sub_radio_question",
      		value=lb_types.Radio(answer=lb_types.ClassificationAnswer(name="first_sub_radio_answer", confidence=0.5))
    )
  ]
)


## NDJSON
bbox_with_radio_subclass_prediction_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "confidence": 0.5,
    "classifications": [{
        "name": "sub_radio_question",
        "confidence": 0.5,
        "answer": 
            { "name":"first_sub_radio_answer", "confidence": 0.5}
         
    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

In [ ]:
########## Polygon ##########
# Python Anotation 
polygon_prediction = lb_types.ObjectAnnotation(
  name = "polygon",  # must match your ontology feature's name 
  confidence = 0.5, 
  value=lb_types.Polygon( # Coordinates for the verticies of your polygon
        points=[lb_types.Point(x=1489.581,y=183.934), lb_types.Point(x=2278.306,y=256.885), lb_types.Point(x=2428.197,y=200.437), lb_types.Point(x=2560.0,y=335.419),
                lb_types.Point(x=2557.386,y=503.165), lb_types.Point(x=2320.596,y=503.103), lb_types.Point(x=2156.083, y=628.943), lb_types.Point(x=2161.111,y=785.519),
                lb_types.Point(x=2002.115, y=894.647), lb_types.Point(x=1838.456,y=877.874), lb_types.Point(x=1436.53,y=874.636), lb_types.Point(x=1411.403,y=758.579),
                lb_types.Point(x=1353.853,y=751.74), lb_types.Point(x=1345.264, y=453.461), lb_types.Point(x=1426.011,y=421.129)]
    ),
)


# NDJSON

polygon_prediction_ndjson = {
  'name': 'polygon',
  'confidence': 0.5,
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

In [ ]:
####### Free text #######
# Confidence is not supported for text prediction
# Python annotation
text_annotation = lb_types.ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=lb_types.Text(answer="sample text")
)

# NDJSON
text_annotation_ndjson = {
  'name': 'free_text',
  'answer': 'sample text',
}

In [ ]:
######### Segmentation mask #########

# Python 
# Identifying what values in the numpy array correspond to the mask annotation
color = (0, 0, 0)

# convert a polygon to mask
im_height, im_width = 100,100 #need to provide the height and width of image.
mask_data = lb_types.MaskData(arr=
                     polygon_prediction.value.draw(height=im_height,width=im_width,color=color))

# convert a 2D array to 3D array
arr_2d = np.zeros((100,100), dtype='uint8')
mask_data = lb_types.MaskData.from_2D_arr(arr_2d)

# a 3D array where 3rd axis is RGB values.
mask_data = lb_types.MaskData(arr= np.zeros([400,450,3],dtype='uint8'))

mask_prediction = lb_types.ObjectAnnotation(
  name = "mask", # must match your ontology feature's name
  confidence=0.5,
  value=lb_types.Mask(mask=mask_data, color=color),
)


# NDJSON
mask_prediction_ndjson = {
  'name': 'mask',
  'confidence': 0.5,
  'classifications': [],
  'mask': {'instanceURI': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA',
  'colorRGB': (0, 0, 0)}
}



In [ ]:
######## Point ########

# Python Annotation
point_prediction = lb_types.ObjectAnnotation(
  name = "point",  # must match your ontology feature's name
  confidence=0.5,
  value = lb_types.Point(x=1166.606, y=1441.768),
)


# NDJSON
point_prediction_ndjson = {
  'name': 'point',
  'confidence': 0.5,
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

In [ ]:
###### Polyline ######


# Python Annotation 

polyline_prediction = lb_types.ObjectAnnotation(
  name = "polyline", # must match your ontology feature's name
  confidence=0.5, ## Not supported for python annotation tools
  value=lb_types.Line( # Coordinates for the keypoints in your polyline
        points=[lb_types.Point(x=2534.353, y=249.471), lb_types.Point(x=2429.492, y=182.092), lb_types.Point(x=2294.322, y=221.962), lb_types.Point(x=2224.491, y=180.463), lb_types.Point(x=2136.123, y=204.716),
                lb_types.Point(x=1712.247, y=173.949), lb_types.Point(x=1703.838, y=84.438), lb_types.Point(x=1579.772, y=82.61), lb_types.Point(x=1583.442, y=167.552),
                lb_types.Point(x=1478.869, y=164.903), lb_types.Point(x=1418.941, y=318.149), lb_types.Point(x=1243.128, y=400.815), lb_types.Point(x=1022.067, y=319.007),
                lb_types.Point(x=892.367, y=379.216), lb_types.Point(x=670.273, y=364.408), lb_types.Point(x=613.114, y=288.16), lb_types.Point(x=377.559, y=238.251),
                lb_types.Point(x=368.087, y=185.064), lb_types.Point(x=246.557, y=167.286), lb_types.Point(x=236.648, y=285.61), lb_types.Point(x=90.929, y=326.412)]
    ),
)

# NDJSON
polyline_prediction_ndjson = {
  'name': 'polyline',
  'confidence':0.5,
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}


## Step 1: Import data rows into Catalog

In [ ]:
# send a sample image as batch to the project
test_img_url = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key": str(uuid.uuid4())
}
dataset = client.create_dataset(name="image_prediction_demo")
data_row = dataset.create_data_row(test_img_url)
print(data_row)

<DataRow {
    "created_at": "2023-02-08 15:34:40+00:00",
    "external_id": null,
    "global_key": "19bdf9f3-fe33-40b5-aa70-8e13dd2896ca",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "uid": "cldvu0iew12rs07v50oys1y4o",
    "updated_at": "2023-02-08 15:34:40+00:00"
}>


## Step 2: Create/select an Ontology for your model predictions
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the name/instructions fields in your annotations to ensure the correct feature schemas are matched.


In [ ]:
ontology_builder = lb.OntologyBuilder(
  classifications=[ # List of Classification objects
    lb.Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=lb.Classification.Type.RADIO,
      instructions="radio_question", 
      options=[
        lb.Option(value="first_radio_answer"),
        lb.Option(value="second_radio_answer")
      ]
    ),
    lb.Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=lb.Classification.Type.CHECKLIST,
      instructions="checklist_question", 
      options=[
        lb.Option(value="first_checklist_answer"),
        lb.Option(value="second_checklist_answer")
      ]
    ), 
    lb.Classification( # Text classification given the name "text"
      class_type=lb.Classification.Type.TEXT,
      instructions="free_text"
    ),
    lb.Classification(
        class_type=lb.Classification.Type.RADIO,
        instructions="nested_radio_question",
        options=[
            lb.Option("first_radio_answer",
                options=[
                    lb.Classification(
                        class_type=lb.Classification.Type.RADIO,
                        instructions="sub_radio_question",
                        options=[lb.Option("first_sub_radio_answer")]
                    )
                ]
            )
          ] 
        ),
    lb.Classification(
      class_type=lb.Classification.Type.CHECKLIST,
      instructions="nested_checklist_question",
      options=[
          lb.Option("first_checklist_answer",
            options=[
              lb.Classification(
                  class_type=lb.Classification.Type.CHECKLIST,
                  instructions="sub_checklist_question", 
                  options=[lb.Option("first_sub_checklist_answer")]
              )
          ]
        )
      ]
    ),      
  ],
  tools=[ # List of Tool objects
    lb.Tool( # Bounding Box tool given the name "box"
      tool=lb.Tool.Type.BBOX,
      name="bounding_box"), 
    lb.Tool( # Bounding Box tool given the name "box"
      tool=lb.Tool.Type.BBOX,
      name="bbox_with_radio_subclass",
      classifications=[
            lb.Classification(
                class_type=lb.Classification.Type.RADIO,
                instructions="sub_radio_question",
                options=[
                  lb.Option(value="first_sub_radio_answer")
                ]
              ),
        ]
      ), 
    lb.Tool( # Polygon tool given the name "polygon"
      tool=lb.Tool.Type.POLYGON,
      name="polygon"),
    lb.Tool( # Segmentation mask tool given the name "mask"
      tool=lb.Tool.Type.SEGMENTATION,
      name="mask"),
 	  lb.Tool( # Point tool given the name "point"
      tool=lb.Tool.Type.POINT,
      name="point"), 
    lb.Tool( # Polyline tool given the name "line"
      tool=lb.Tool.Type.LINE,
      name="polyline")]
)

ontology = client.create_ontology("Image Prediction Import Demo", ontology_builder.asdict(), media_type=lb.MediaType.Image)

## Step 3: Create a Model and Model Run

In [ ]:
# create Model
model = client.create_model(name="image_model_run_" + str(uuid.uuid4()),
                            ontology_id=ontology.uid)
# create Model Run
model_run = model.create_model_run("iteration 1")

## Step 4: Send data rows to the Model Run

In [ ]:
model_run.upsert_data_rows([data_row.uid])

True

## Step 5. Create the predictions payload

Create the prediction payload using the snippets of code in ***Supported Predictions*** section. 

The resulting label_ndjson should have exactly the same content for predictions that are supported by both (with exception of the uuid strings that are generated)

In [ ]:
# Create a Label for predictions
label_prediction = lb_types.Label(
    data=lb_types.ImageData(uid=data_row.uid),
    annotations = [
      radio_prediction,
      checklist_prediction, 
      bbox_prediction, 
      bbox_with_radio_subclass_prediction, 
      polyline_prediction,
      polygon_prediction, 
      mask_prediction, 
      point_prediction,
      text_annotation
      ]
)

# Create a label list 
label_list_prediction = [label_prediction]

# Convert the prediction label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
ndjson_prediction = list(lb_serializers.NDJsonConverter.serialize(label_list_prediction))

If using NDJSON:

In [ ]:
ndjson_prediction_method2 = []

for annot in [
    radio_prediction_ndjson,
    checklist_prediction_ndjson, 
    bbox_prediction_ndjson, 
    bbox_with_radio_subclass_prediction_ndjson, 
    polygon_prediction_ndjson, 
    mask_prediction_ndjson, 
    point_prediction_ndjson,
    polyline_prediction_ndjson,
    text_annotation_ndjson, 
    nested_radio_prediction_ndjson,
    nested_checklist_prediction_ndjson
]:
  annot.update({
      'uuid': str(uuid.uuid4()),
      'dataRow': {'id': data_row.uid},
  })
  ndjson_prediction_method2.append(annot)

## Step 6. Upload the predictions payload to the Model Run 

In [ ]:
# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job"+str(uuid.uuid4()),
    predictions=ndjson_prediction_method2)

# Errors will appear for prediction uploads that failed.
print("Errors:",  upload_job_prediction.errors)

Errors: []


## Step 7: Send annotations to a model run
To visualize both annotations and predictions in the model run we will create a project with ground truth annotations. 
To send annotations to a Model Run, we must first import them into a project, create a label payload and then send them to the Model Run.

##### 7.1. Create a labelbox project

In [ ]:
# Create a Labelbox project
project = client.create_project(name="image_prediction_demo",
                                    # Quality Settings setup 
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=lb.MediaType.Image)
project.setup_editor(ontology)

##### 7.2. Create a batch to send to the project 

In [ ]:
project.create_batch(
  "batch_predictions_demo", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

<Batch ID: 2374e520-a7c6-11ed-8123-cf5deef7e8cf>

##### 7.3 Create the annotations payload

In [ ]:
########### Annotations ###########
radio_annotation_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer'}
} 

nested_radio_annotation_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer"},
  "classifications" : [
   {'name': 'sub_radio_question', 'answer': {'name': 'first_sub_radio_answer'}}
   ]
}

checklist_annotation_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'}
  ]
}

bbox_annotation_ndjson = {
  'name': 'bounding_box',
  'bbox': {
          "top": 977,
          "left": 1690,
          "height": 330,
          "width": 225
      }
}

bbox_with_radio_subclass_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "classifications": [{
        "name": "sub_radio_question",
        "answer": 
            { "name":"first_sub_radio_answer" }
         
    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

polygon_annotation_ndjson = {
  'name': 'polygon',
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

mask_annotation_ndjson = {
  'name': 'mask',
  'classifications': [],
  'mask': {'instanceURI': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA',
  'colorRGB': (0, 0, 0)}
}


point_annotation_ndjson = {
  'name': 'point',
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

point_annotation_ndjson = {
  'name': 'point',
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

polyline_annotation_ndjson = {
  'name': 'polyline',
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}

nested_checklist_annotation_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer", 
      "classifications" : [
        {
          "name": "sub_checklist_question", 
          "answer": {"name": "first_sub_checklist_answer"}
        }          
      ]         
  }]
}

text_annotation_ndjson = {
  'name': 'free_text',
  'answer': 'sample text',
}


##### 7.4. Create the label object

In [ ]:
# Create a Label object by identifying the applicable data row in Labelbox and providing a list of annotations
ndjson_annotation = []
for annot in [
    radio_annotation_ndjson, 
    checklist_annotation_ndjson, 
    bbox_annotation_ndjson, 
    bbox_with_radio_subclass_ndjson, 
    polygon_annotation_ndjson, 
    mask_annotation_ndjson, 
    point_annotation_ndjson, 
    polyline_annotation_ndjson,
    nested_radio_annotation_ndjson,
    nested_checklist_annotation_ndjson,
    text_annotation_ndjson
]:
  annot.update({
      'uuid': str(uuid.uuid4()),
      'dataRow': {'id': data_row.uid},
  })
  ndjson_annotation.append(annot) 



##### 7.5. Upload annotations to the project using Label Import

In [ ]:
upload_job_annotation = lb.LabelImport.create_from_objects(
    client = client,
    project_id = project.uid,
    name="annotation_import_" + str(uuid.uuid4()),
    labels=ndjson_annotation)

upload_job_annotation.wait_until_done()
# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_annotation.errors)


Errors: []


##### 7.6 Send the annotations to the Model Run

In [ ]:
# get the labels id from the project
label_ids = [x['ID'] for x in project.export_labels(download=True)]
model_run.upsert_labels(label_ids)

True

## Optional deletions for cleanup 


In [ ]:
# project.delete()
# dataset.delete()